<a href="https://colab.research.google.com/github/JihoonPark99/NLP_study/blob/main/_13_%EA%B0%9C%EC%B2%B4%EB%AA%85_%EC%9D%B8%EC%8B%9D(Named_Entity_Recognition).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 개체명 인식(Named Entity Recognition)


* 개체명 인식은 텍스트에서 이름을 가진 개체를 인식하는 기술      
* 가령, '철수와 영희는 밥을 먹었다'에서 이름과 사물을 추출하는 개체명 인식 모델 결과 

  철수 - 이름    
  영희 - 이름    
  밥 - 사물

## 개체명 인식 - NLTK

* https://wikidocs.net/30682

* `nltk` 라이브러리에서는 미리 학습된 개체명 인식 모델을 제공

### 라이브러리 준비

In [ ]:
import nltk

nltk.download('words')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.


True

### 토큰화 및 품사 태깅

In [ ]:
from nltk import word_tokenize, pos_tag, ne_chunk

sentence = "James is working at Disney in London"
sentence = pos_tag(word_tokenize(sentence))
print(sentence)

[('James', 'NNP'), ('is', 'VBZ'), ('working', 'VBG'), ('at', 'IN'), ('Disney', 'NNP'), ('in', 'IN'), ('London', 'NNP')]


### 개체명 인식

In [ ]:
sentence = ne_chunk(sentence)
##ne_chunk를 이용해서 개체명이 나옴

# PERSON
# ORGANIZATION
# GPE : 국가
print(sentence)

(S
  (PERSON James/NNP)
  is/VBZ
  working/VBG
  at/IN
  (ORGANIZATION Disney/NNP)
  in/IN
  (GPE London/NNP))


개체명 인식이 되는 딥러닝 모델을 직접 만들어보자

## 개체명 인식 - LSTM

* https://wikidocs.net/24682

* 사용자가 제공되고 있는 개체명 인식 모델과는 다른 개체명을 정의해 사용하는 것이 필요할 수 있음
* 직접 개체명 인식 모델을 구성해 학습하고 사용할 수 있음

### 라이브러리 준비

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

import numpy as np
import urllib.request

### 데이터 준비

* 공개된 개체명 인식 데이터셋을 이용
  + https://raw.githubusercontent.com/Franck-Dernoncourt/NeuroNER/master/neuroner/data/conll2003/en/train.txt
* 해당 데이터는 단어-개체명 형식으로 이루어져 있으므로 이를 가공해 데이터셋을 생성

In [ ]:
tagged_sentences = []
sentence = []

with urllib.request.urlopen('https://raw.githubusercontent.com/Franck-Dernoncourt/NeuroNER/master/neuroner/data/conll2003/en/train.txt') as f:
  for line in f: ##라인단위로 읽어오기
    line = line.decode('utf-8') ##디코딩해주기
    if len(line) == 0 or line.startswith('-DOCSTART') or line[0] == "\n":
      if len(sentence) > 0:
        tagged_sentences.append(sentence)
        sentence = []
      ##아닌경우는 continue
      continue
    ##정상일 경우는 
    splits = line.strip().split(' ') #스페이스 기준으로 split
    word = splits[0].lower()
    sentence.append([word, splits[-1]])

print(len(tagged_sentences))
print(tagged_sentences[0])




14041
[['eu', 'B-ORG'], ['rejects', 'O'], ['german', 'B-MISC'], ['call', 'O'], ['to', 'O'], ['boycott', 'O'], ['british', 'B-MISC'], ['lamb', 'O'], ['.', 'O']]


[]

### 데이터 전처리

* 단어와 개체명 태그를 분리해서 데이터를 구성

In [ ]:
sentences, ner_tags = [], []
for tagged_sentence in tagged_sentences:
  sentence, tag_info = zip(*tagged_sentence) 
  ##tagged_sentences에서 하나씩 가져와서 분류하기
  sentences.append(list(sentence)) #단어정보만 
  ner_tags.append(list(tag_info))  #개체명 태그정보만 있음


* 정제 및 빈도 수가 높은 상위 단어들만 추출하기 위해 토큰화 작업

In [ ]:
## 토크나이저 지정

#train tokenizer
max_words = 4000
src_tokenizer = Tokenizer(num_words = max_words, oov_token='OOV')
src_tokenizer.fit_on_texts(sentences)

#target tokenizer
tar_tokenizer = Tokenizer()
tar_tokenizer.fit_on_texts(ner_tags)

In [ ]:
tar_tokenizer.word_index

{'o': 1,
 'b-loc': 2,
 'b-per': 3,
 'b-org': 4,
 'i-per': 5,
 'i-org': 6,
 'b-misc': 7,
 'i-loc': 8,
 'i-misc': 9}

In [ ]:
vocab_size = max_words
tag_size = len(tar_tokenizer.word_index) + 1 #전체 타겟의 토크나이저의 인덱스 +1
#tar_tokenizer.word_index 바로 위에서 확인

print(vocab_size)
print(tag_size)

4000
10


* 데이터를 학습에 활용하기 위해 데이터를 배열로 변환
* 해당 작업은 토큰화 툴의 `texts_to_sequences()`를 통해 수행

In [ ]:
X_train = src_tokenizer.texts_to_sequences(sentences) ##단어들
y_train = tar_tokenizer.texts_to_sequences(ner_tags) ##개체명
print('X_train.shape : ', len(X_train))
print('y_train.shape : ', len(y_train))

X_train.shape :  14041
y_train.shape :  14041


In [ ]:
sentences[0], ner_tags[0]

(['eu', 'rejects', 'german', 'call', 'to', 'boycott', 'british', 'lamb', '.'],
 ['B-ORG', 'O', 'B-MISC', 'O', 'O', 'O', 'B-MISC', 'O', 'O'])

In [ ]:
X_train[0], y_train[0]

([989, 1, 205, 629, 7, 3939, 216, 1, 3], [4, 1, 7, 1, 1, 1, 7, 1, 1])

In [ ]:
print(len(X_train))
print(len(y_train))

14041
14041


패딩을 하자자
* 학습에 투입할 때는 동일한 길이를 가져야 하므로, 지정해둔 최대 길이에 맞춰 모든 데이터를 동일한 길이로 맞춰줌
* 일반적으로 길이를 맞출 때는 모자란 길이만큼 0을 추가


In [ ]:
max_len = 70
X_train = pad_sequences(X_train, padding='post', maxlen = max_len)
##패딩을 뒤에다 붙임 : post

y_train = pad_sequences(y_train, padding='post', maxlen= max_len)

print('X_train크기 : ', X_train.shape)
print('y_train크기 : ', y_train.shape)

X_train크기 :  (14041, 70)
y_train크기 :  (14041, 70)


In [ ]:
X_train[0], y_train[0] ##padding='post'를 통해서 다 뒤에붙은걸 알 수 있음.

(array([ 989,    1,  205,  629,    7, 3939,  216,    1,    3,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0], dtype=int32),
 array([4, 1, 7, 1, 1, 1, 7, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0], dtype=int32))

* 훈련, 실험 데이터 분리 및 원 핫 인코딩을 시행

In [ ]:
tag_size

10

In [ ]:
## X_train -> X_train, X_test로 분리 = 5:1
## y_train -> y_train, y_test로 분리리 = 5:1
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=111)

print('X_train.shape :', X_train.shape) #  (11232, 70)
print('X_test.shape' ,X_test.shape) # (2809, 70)
print('y_train.shape : ',y_train.shape) #  (11232, 70)
print('y_test.shape : ',y_test.shape) #  (2809, 70)
print('\n')
print('y_train[0] : ', y_train[0]) 
#  [3 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
#   0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]

print('y_test[0] : ',y_test[0])
#  [1 1 1 1 1 1 3 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
#   0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


y_train = to_categorical(y_train, num_classes=tag_size) 
#y_train이 [3,1,0] -> [[0,0,0,1,0,0,0,0,0,0], #원-핫-인코딩
#                      [0,1,0,0,0,0,0,0,0,0],
#                      [0,0,0,1,0,0,0,0,0,0]] 이런 식으로 바꾸기
y_test = to_categorical(y_test, num_classes=tag_size)
print('\n')
print('after categorical y_train[0]: ', y_train[0]) 
print('after categorical y_test[0]: ', y_test[0])


X_train.shape : (11232, 70)
X_test.shape (2809, 70)
y_train.shape :  (11232, 70)
y_test.shape :  (2809, 70)


y_train[0] :  [3 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
y_test[0] :  [1 1 1 1 1 1 3 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


after categorical y_train[0]:  [[0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.

In [ ]:
print(X_train.shape) #(11232, 70)
print(y_train.shape) #(11232, 70, 10) #
print(X_test.shape) #(2809, 70)
print(y_test.shape) #(2809, 70, 10)

(11232, 70)
(11232, 70, 10)
(2809, 70)
(2809, 70, 10)


- padding때문에 70이 되고,
- y_train과 y_test는 tagging때문에 10이 들어가 있다. 

* 최종적으로 생성된 데이터셋의 크기는 다음과 같음
* 이제야 학습을 돌릴 준비 완료!!!

### 모델 구축 및 학습

* 모델 구축에는 `keras`를 이용
* 해당 작업에 필요한 함수들을 추가로 import

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, Bidirectional, TimeDistributed
from keras.optimizers import Adam

모델의 구성

1. 입력을 실수 벡터로 임베딩
2. 양방향 LSTM 구성
3. Dense layer를 통한 각 태그에 속할 확률 예측

`TimeDistributed`는 상위 layer의 출력이 step에 따라 여러 개로 출력되어 이를 적절하게 분배해주는 역할


양방향 LSTM(다 대 다 구조의 LSTM, Bi-directional LSTM)
- 

timedistributed역할 구글링
- https://m.blog.naver.com/PostView.naver?isHttpsRedirect=true&blogId=acelhj&logNo=221653390039

- Keras 도큐멘트에 따르면 TimeDistributed는 시간의 조각(Temporal slice)를 만들어 준다.
2.0에서는 그냥 Dense를 써도 된다.

- (sample, time, width, length, channel) 를 TimeDistributed 를 사용하여 시간 차원을 따라서 (sample, width, length, channel) 로 만든다.

In [ ]:
vocab_size, max_len

(4000, 70)

In [ ]:
#####모델구성########
model = Sequential()

##처음에는 Embedding을 넣어줘야한다.
model.add(Embedding(input_dim=vocab_size, output_dim=128, input_length=max_len, mask_zero=True))
##make_zero=True :  숫자 0은 연산에서 제외시킨다는 옵션
###주로 padding을 해서 0이 많아졌을때 사용.

##Bidirectional
model.add(Bidirectional(LSTM(256, return_sequences=True)))

model.add(TimeDistributed(Dense(tag_size, activation='softmax'))) ##다중분류문제랑 똑같음

model.summary()








Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 70, 128)           512000    
                                                                 
 bidirectional (Bidirectiona  (None, 70, 512)          788480    
 l)                                                              
                                                                 
 time_distributed (TimeDistr  (None, 70, 10)           5130      
 ibuted)                                                         
                                                                 
Total params: 1,305,610
Trainable params: 1,305,610
Non-trainable params: 0
_________________________________________________________________


- 임베딩 사이즈 : 128
- 파라미터 수도 꽤 된다. 
  - Bidirectional을 사용했기 때문에!!
- 양방향 LSTM은 모든 시점에 대해서 개체명 레이블 개수만큼의 선택지 중 하나를 예측하는 다중 클래스 분류 문제를 수행하는 모델


* 모델 컴파일 및 학습 진행, 평가

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(0.001),
              metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=128, epochs=3, validation_data=(X_test, y_test))

Epoch 1/3
88/88 [==============================] - 175s 2s/step - loss: 0.1855 - accuracy: 0.8254 - val_loss: 0.1281 - val_accuracy: 0.8330
Epoch 2/3
88/88 [==============================] - 151s 2s/step - loss: 0.0986 - accuracy: 0.8555 - val_loss: 0.0773 - val_accuracy: 0.8816
Epoch 3/3
88/88 [==============================] - 149s 2s/step - loss: 0.0661 - accuracy: 0.9024 - val_loss: 0.0567 - val_accuracy: 0.9181


In [ ]:
model.evaluate(X_test, y_test)

88/88 [==============================] - 17s 188ms/step - loss: 0.0567 - accuracy: 0.9181


[0.05671876668930054, 0.9181334972381592]

### 학습한 모델을 통한 예측

* 예측을 확인하기 위해서 인덱스를 단어로 변환해줄 사전이 필요
* 사전은 토큰화 툴의 사전을 이용

In [ ]:
idx2word = src_tokenizer.index_word
idx2ner = tar_tokenizer.index_word
idx2ner[0] = 'PAD'#'PAD' : 패딩된 툴

* 예측 시각화

In [ ]:
np.argmax(y_test[10],-1)

array([3, 5, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0])

In [ ]:

i = 10 #10개를 보자
y_predicted = model.predict(np.array([X_test[i]])) #10번째 predict값
print(y_predicted)

y_predicted = np.argmax(y_predicted, axis=-1)
print('after argmax : ', y_predicted)

true = np.argmax(y_test[i], -1)
print('정답 개체명 :', true)

print('출력 : {:15}:{:15}:{}'.format("단어", "실제값", "예측값"))
print('='*35)

for w,t,pred in zip(X_test[i], true, y_predicted[0]):
  if w != 0 :
    print("{:17} : {:7} : {}".format(idx2word[w], idx2ner[t].upper(), idx2ner[pred].upper()))

  ##idx2word[w] :
  ##idx2ner[t] : 해당하는 태깅값값
  ##idx2ner[pred] : 예측한 개체명

1/1 [==============================] - 4s 4s/step
[[[2.16101267e-04 2.03590080e-01 4.10742611e-02 1.67077392e-01
   3.26568604e-01 6.76828921e-02 1.03821427e-01 6.45565018e-02
   1.04819946e-02 1.49308210e-02]
  [3.05787602e-04 7.45386630e-02 1.42279327e-01 4.26489040e-02
   2.83528626e-01 1.30194858e-01 1.61911160e-01 1.09455086e-01
   3.34853306e-02 2.16522403e-02]
  [3.35136901e-05 7.38236725e-01 3.75819975e-03 3.98812629e-03
   3.04372385e-02 5.61094331e-03 1.73101276e-01 1.31761963e-02
   9.72446799e-03 2.19333321e-02]
  [3.65834348e-05 1.07065670e-03 8.30523670e-01 1.67304482e-02
   4.90313955e-02 5.15993778e-03 1.60587914e-02 5.99287674e-02
   1.66979209e-02 4.76173824e-03]
  [3.65770035e-07 9.56261337e-01 4.51412998e-05 2.66820516e-05
   3.56527598e-04 1.57622341e-03 2.98951548e-02 8.22482107e-05
   3.74200614e-03 8.01431295e-03]
  [2.82172063e-09 9.99570787e-01 2.56252247e-06 1.37695361e-06
   5.34430910e-05 7.20287176e-07 2.97316205e-04 4.93455582e-06
   5.14613657e-06 6.3669